# Set up

In [1]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Creating and running a Graph

In [3]:
import tensorflow as tf
import numpy as np
reset_graph()
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [4]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [6]:
sess.close()

In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [8]:
print(result)

42


In [9]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [10]:
print(result)

42


In [11]:
sess.close()

# Managing graphs

In [12]:
reset_graph()
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [13]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

In [15]:
w = tf.constant(3)
x = w + 3
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

11
18


In [16]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

11
18


# Linear Regression

## Prepare

In [17]:
def fetch_caifornia_housing():
    import pandas as pd
    from sklearn.preprocessing import Imputer
    file_path='datasets/housing/housing.csv'
    data_df = pd.read_csv(file_path)
    data_attribs= ['longitude','latitude','housing_median_age',
                  'total_rooms','total_bedrooms','population',
                  'households','median_income']
    label_attibs = ['median_house_value']
    imputer = Imputer(strategy='median')
    return imputer.fit_transform(data_df[data_attribs].values), data_df[label_attibs].values

In [19]:
reset_graph()
data, target = fetch_caifornia_housing()
data.shape, target.shape

((20640, 8), (20640, 1))

In [20]:
m, n = data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [21]:
theta_value

array([[ -3.62829275e+06],
       [ -4.32717031e+04],
       [ -4.30518516e+04],
       [  1.13395923e+03],
       [ -6.47900295e+00],
       [  8.14451599e+01],
       [ -3.98782883e+01],
       [  7.81404572e+01],
       [  3.96517695e+04]], dtype=float32)

In [23]:
X = housing_data_plus_bias
y = target.reshape(-1,1)
theta_numpy  = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print(theta_numpy)

[[ -3.57011806e+06]
 [ -4.26104026e+04]
 [ -4.24754782e+04]
 [  1.14445085e+03]
 [ -6.62091740e+00]
 [  8.11609666e+01]
 [ -3.98732002e+01]
 [  7.93047225e+01]
 [  3.97522237e+04]]


In [24]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(data, target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [25]:
print(np.r_[lin_reg.intercept_.reshape(-1,1), lin_reg.coef_.T])

[[ -3.57011806e+06]
 [ -4.26104026e+04]
 [ -4.24754782e+04]
 [  1.14445085e+03]
 [ -6.62091740e+00]
 [  8.11609666e+01]
 [ -3.98732002e+01]
 [  7.93047225e+01]
 [  3.97522237e+04]]


# Using Batch Gradient Descent

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scale_housing_data = scaler.fit_transform(data)
scale_housing_data_bias = np.c_[np.ones((m,1)), scale_housing_data]

## Manually computing the gradients

In [27]:
reset_graph()
n_epochs = 10000
learning_rate = 0.01
X = tf.constant(scale_housing_data_bias, dtype=tf.float32, name='X')
y = tf.constant(target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
traning_op = tf.assign(theta, theta-learning_rate*gradients)

In [28]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print("Epoch", epoch, 'MSE', mse.eval())
        sess.run(traning_op)
    best_theta=theta.eval()

Epoch 0 MSE 5.61048e+10
Epoch 100 MSE 6.81554e+09
Epoch 200 MSE 5.62697e+09
Epoch 300 MSE 5.3998e+09
Epoch 400 MSE 5.25044e+09
Epoch 500 MSE 5.1438e+09
Epoch 600 MSE 5.06695e+09
Epoch 700 MSE 5.0112e+09
Epoch 800 MSE 4.97055e+09
Epoch 900 MSE 4.94075e+09
Epoch 1000 MSE 4.9188e+09
Epoch 1100 MSE 4.90256e+09
Epoch 1200 MSE 4.89049e+09
Epoch 1300 MSE 4.88147e+09
Epoch 1400 MSE 4.87472e+09
Epoch 1500 MSE 4.86962e+09
Epoch 1600 MSE 4.86576e+09
Epoch 1700 MSE 4.86282e+09
Epoch 1800 MSE 4.86057e+09
Epoch 1900 MSE 4.85884e+09
Epoch 2000 MSE 4.8575e+09
Epoch 2100 MSE 4.85646e+09
Epoch 2200 MSE 4.85565e+09
Epoch 2300 MSE 4.855e+09
Epoch 2400 MSE 4.8545e+09
Epoch 2500 MSE 4.8541e+09
Epoch 2600 MSE 4.85377e+09
Epoch 2700 MSE 4.85351e+09
Epoch 2800 MSE 4.85331e+09
Epoch 2900 MSE 4.85314e+09
Epoch 3000 MSE 4.853e+09
Epoch 3100 MSE 4.85288e+09
Epoch 3200 MSE 4.85279e+09
Epoch 3300 MSE 4.85271e+09
Epoch 3400 MSE 4.85266e+09
Epoch 3500 MSE 4.8526e+09
Epoch 3600 MSE 4.85255e+09
Epoch 3700 MSE 4.85251e+0

In [29]:
best_theta

array([[ 206855.4375    ],
       [ -85352.828125  ],
       [ -90710.125     ],
       [  14403.05273438],
       [ -14418.48242188],
       [  33898.39453125],
       [ -45185.1484375 ],
       [  30463.27148438],
       [  75513.5859375 ]], dtype=float32)

### using autodiff

In [30]:
reset_graph()
n_epochs = 10000
learning_rate = 0.01
X = tf.constant(scale_housing_data_bias, dtype=tf.float32, name='X')
y = tf.constant(target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
traning_op = tf.assign(theta, theta-learning_rate*gradients)

In [31]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print("Epoch", epoch, 'MSE', mse.eval())
        sess.run(traning_op)
    best_theta=theta.eval()

Epoch 0 MSE 5.61048e+10
Epoch 100 MSE 6.81554e+09
Epoch 200 MSE 5.62697e+09
Epoch 300 MSE 5.3998e+09
Epoch 400 MSE 5.25044e+09
Epoch 500 MSE 5.1438e+09
Epoch 600 MSE 5.06695e+09
Epoch 700 MSE 5.0112e+09
Epoch 800 MSE 4.97055e+09
Epoch 900 MSE 4.94075e+09
Epoch 1000 MSE 4.9188e+09
Epoch 1100 MSE 4.90256e+09
Epoch 1200 MSE 4.89049e+09
Epoch 1300 MSE 4.88147e+09
Epoch 1400 MSE 4.87472e+09
Epoch 1500 MSE 4.86962e+09
Epoch 1600 MSE 4.86576e+09
Epoch 1700 MSE 4.86282e+09
Epoch 1800 MSE 4.86057e+09
Epoch 1900 MSE 4.85884e+09
Epoch 2000 MSE 4.8575e+09
Epoch 2100 MSE 4.85646e+09
Epoch 2200 MSE 4.85565e+09
Epoch 2300 MSE 4.855e+09
Epoch 2400 MSE 4.8545e+09
Epoch 2500 MSE 4.8541e+09
Epoch 2600 MSE 4.85377e+09
Epoch 2700 MSE 4.85351e+09
Epoch 2800 MSE 4.85331e+09
Epoch 2900 MSE 4.85314e+09
Epoch 3000 MSE 4.853e+09
Epoch 3100 MSE 4.85288e+09
Epoch 3200 MSE 4.85279e+09
Epoch 3300 MSE 4.85271e+09
Epoch 3400 MSE 4.85265e+09
Epoch 3500 MSE 4.8526e+09
Epoch 3600 MSE 4.85255e+09
Epoch 3700 MSE 4.85251e+0

In [32]:
best_theta

array([[ 206855.4375    ],
       [ -85352.84375   ],
       [ -90710.140625  ],
       [  14403.04980469],
       [ -14418.47070312],
       [  33898.37890625],
       [ -45185.15234375],
       [  30463.27539062],
       [  75513.578125  ]], dtype=float32)

In [33]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z+i) + z* np.sin(b-i)
    return z

In [34]:
reset_graph()
a = tf.Variable(0.2, name='a')
b = tf.Variable(0.3, name='b')
z = tf.Variable(0.0, name='z0')
for i in range(100):
    z = a * tf.cos(z+i) + z*tf.sin(b-i)
grads = tf.gradients(z, [a,b])
init = tf.global_variables_initializer()

In [35]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.212537
[-1.1388494, 0.19671395]


### Using a GradientDescentOptimizer

In [40]:
reset_graph()
n_epochs = 1000
learning_rate = 0.001
X = tf.Variable(scale_housing_data_bias, dtype=tf.float32, name='X')
y = tf.Variable(target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [41]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
traing_op = optimizer.minimize(mse)

In [42]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(traing_op)
    best_theta = theta.eval()
print(best_theta)

Epoch 0 MSE= 5.61048e+10
Epoch 100 MSE= nan
Epoch 200 MSE= nan
Epoch 300 MSE= nan
Epoch 400 MSE= nan
Epoch 500 MSE= nan
Epoch 600 MSE= nan
Epoch 700 MSE= nan
Epoch 800 MSE= nan
Epoch 900 MSE= nan
[[ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]


# Feeding data to the training algorithm

## Palceholder nodes

In [43]:
reset_graph()
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})

In [44]:
B_val_1

array([[ 6.,  7.,  8.]], dtype=float32)

In [45]:
B_val_2

array([[  9.,  10.,  11.],
       [ 12.,  13.,  14.]], dtype=float32)

# Name scopes

In [46]:
reset_graph()
a1 = tf.Variable(0, name='a')
a2 = tf.Variable(0, name='a')
with tf.name_scope('param'):
    a3 = tf.Variable(0, name='a')
with tf.name_scope('param'):
    a4 = tf.Variable(0, name='a')
for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


# Modularity

In [47]:
reset_graph()
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
w1 = tf.Variable(tf.random_normal((n_features, 1)), name='weights1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name='weights2')
b1 = tf.Variable(0.0, name='bias1')
b2 = tf.Variable(0.0, name='bias2')
z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')
relu1 = tf.maximum(z1, 0, name='relu1')
relu2 = tf.maximum(z2, 0, name='relu2')
ouput = tf.add(relu1, relu2, name='output')

In [51]:
reset_graph()
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0., name='relu')

In [52]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
ouput = tf.add_n(relus, name='output')

In [53]:
file_writer = tf.summary.FileWriter('logs/relu1', tf.get_default_graph())

In [54]:
file_writer.close()

# Sharing Variables

In [56]:
reset_graph()
def relu(X, threshold):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, 0., name='max')

In [57]:
threshold = tf.Variable(0.0, name='threshold')
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X, threshold) for _ in range(5)]
output = tf.add_n(relus, name='output')

In [59]:
reset_graph()
with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold', shape=(),
                               initializer=tf.constant_initializer(0.0))

In [60]:
with tf.variable_scope('relu', reuse=True):
    threshold =tf.get_variable('threshold')

In [61]:
with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold = tf.get_variable('threshold')